In [4]:
#Loading packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
import re
#import scrape_functions_un as kzd
import scraping_model as sm 

## Players Dictionary

In [5]:
players_names = {'Rory McIlroy': '17419',
            'Tiger Woods': '10085', 
            'Jordan Spieth': '19976',
            'Justin Thomas': '20037',
            'Scottie Scheffler': '25786',
            'Collin Morikawa': '33992',
            'Sergio García': '10170',
            'Dustin Johnson': '17976',
            'Phil Mickelson': '10083',
            'Ian Poulter': '11956',
            'Talor Gooch': '27168',
            'Cameron Smith': '27755',
            'Joaquin Niemann': '33997',
            'Bryson DeChambeau': '29148',
            'Brooks Koepka': '25730',
            'Jon Rahm': '28177'}

### We want to get all the dates and urls for each of the player's pages

In [6]:
niemann_url = 'https://www.asapsports.com/show_player.php?id=33997'

response = requests.get(niemann_url)
soup = BeautifulSoup(response.text, 'html.parser')
dates_niemann = soup.find_all('b')

In [7]:
## Retrieving player interview links
links = []
for row in dates_niemann:
        a_tags = soup.find_all('a', href=True)
        
for atag in a_tags:
    url = atag['href']
    if url.startswith('http://www.asapsports.com/show_interview.php?id='):
         links.append(url)

In [14]:

URL = 'https://www.asapsports.com/show_interview.php?id=194453'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Getting Data
event = soup.find('h1')
dates = soup.find('h2')
names_soup = soup.find_all('h3')


names = []
for i in names_soup:
    if i.text in players_names:
        names.append(i.text)




print(dates.text)
print(event.text)
print(names)


March 1, 2024
LIV GOLF JEDDAH
['Jon Rahm', 'Joaquin Niemann']


In [15]:
dates, answers = sm.parse_interview(URL)
niemann_result = pd.DataFrame({'date':dates,'answer':answers})

In [16]:
niemann_result

,date,answer
0,"March 1, 2024","(jon rahm, [really, really, good, day, drove, ..."
1,"March 1, 2024","(jon rahm, [like, everything, there's, going, ..."
2,"March 1, 2024","(joaquin niemann, [yeah, it, was, a, good, day..."
3,"March 1, 2024","(joaquin niemann, [yeah, i, think, it's, been,..."
4,"March 1, 2024","(jon rahm, [yeah, i, think, what, were, we, un..."
5,"March 1, 2024","(joaquin niemann, [yeah, i, think, i've, been,..."
6,"March 1, 2024","(joaquin niemann, [normally, it's, coming, fro..."
7,"March 1, 2024","(jon rahm, [i, felt, horrible, on, the, tee, b..."
8,"March 1, 2024","(jon rahm, [saying, that, i, don't, have, much..."
9,"March 1, 2024","(jon rahm, [that's, not, really, true, i've, b..."


In [17]:
niemann_result['player'] = [i for i, n in niemann_result['answer']]
niemann_result['answer'] = [n for i, n in niemann_result['answer']]
niemann_result

,date,answer,player
0,"March 1, 2024","[really, really, good, day, drove, it, great, ...",jon rahm
1,"March 1, 2024","[like, everything, there's, going, to, be, a, ...",jon rahm
2,"March 1, 2024","[yeah, it, was, a, good, day, since, the, star...",joaquin niemann
3,"March 1, 2024","[yeah, i, think, it's, been, good, but, i, jus...",joaquin niemann
4,"March 1, 2024","[yeah, i, think, what, were, we, under, as, a,...",jon rahm
5,"March 1, 2024","[yeah, i, think, i've, been, really, good, lat...",joaquin niemann
6,"March 1, 2024","[normally, it's, coming, from, the, north, nor...",joaquin niemann
7,"March 1, 2024","[i, felt, horrible, on, the, tee, because, obv...",jon rahm
8,"March 1, 2024","[saying, that, i, don't, have, much, experienc...",jon rahm
9,"March 1, 2024","[that's, not, really, true, i've, been, part, ...",jon rahm


In [18]:
def get_questions(bold_tags):
    questions = []
    for tag in bold_tags:
        if tag.text.startswith('Q.'):
            parent_p = tag.find_parent('p')
            if parent_p:
                question_text = parent_p.get_text(strip=True)
                questions.append(question_text)
    return questions

In [19]:
bold_text = soup.find_all('b')
questions = get_questions(bold_text) 
stripped_questions = [question.strip('Q.') for question in questions]
stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

# Si no hay pregunta para la respuesta, quiero que rellenes con 'MODERATOR' 

since the moderator questions don't have a fixed pattern, we struggle to retrieve them, we will impute them with 'moderator' in the question; in the text analysis we might get rid of the token so it doesn't consider it an important token.

In [13]:
num_to_fill = len(niemann_result['answer']) - len(stripped_questions)
stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions

niemann_result['question'] = stripped_questions

In [48]:
niemann_result['Ans LIV?'] =niemann_result['answer'].apply(lambda x: 1 if 'liv' in x else 0)
niemann_result['Ans PGA?'] =niemann_result['answer'].apply(lambda x: 1 if 'pga' in x else 0)
niemann_result['Q LIV?'] =niemann_result['question'].apply(lambda x: 1 if 'liv' in x else 0)
niemann_result['Q PGA?'] =niemann_result['question'].apply(lambda x: 1 if 'pga' in x else 0)


In [14]:
niemann_result

,date,answer,player,question
0,"March 1, 2024","[really, really, good, day, drove, it, great, ...",jon rahm,MODERATOR
1,"March 1, 2024","[like, everything, there's, going, to, be, a, ...",jon rahm,"[most, of, the, guys, talk, about, how, many, ..."
2,"March 1, 2024","[yeah, it, was, a, good, day, since, the, star...",joaquin niemann,"[joaco, another, great, round, under, par, tie..."
3,"March 1, 2024","[yeah, i, think, it's, been, good, but, i, jus...",joaquin niemann,"[joaco, obviously, the, last, month, has, been..."
4,"March 1, 2024","[yeah, i, think, what, were, we, under, as, a,...",jon rahm,"[jon, talk, about, the, group, that, you, guys..."
5,"March 1, 2024","[yeah, i, think, i've, been, really, good, lat...",joaquin niemann,"[joaco, it's, a, great, set, of, stats, you're..."
6,"March 1, 2024","[normally, it's, coming, from, the, north, nor...",joaquin niemann,"[joaco, all, week, on, the, range, it's, been,..."
7,"March 1, 2024","[i, felt, horrible, on, the, tee, because, obv...",jon rahm,"[jon, who, was, happier, on, you, or, phil, be..."
8,"March 1, 2024","[saying, that, i, don't, have, much, experienc...",jon rahm,"[obviously, with, the, team, event, it's, new,..."
9,"March 1, 2024","[that's, not, really, true, i've, been, part, ...",jon rahm,"[in, the, golf, world]"


In [ ]:
URL_base = 'http://www.asapsports.com/show_interview.php?id='
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Getting Data
names = soup.find_all('h1')
dates = soup.find('h2')
event = soup.find('h3')


print(names[i].text for i in names)
print(dates.text)
print(event.text)


links = []
for row in dates_niemann:
        a_tags = soup.find_all('a', href=True)
        
for atag in a_tags:
    url = atag['href']
    if url.startswith('http://www.asapsports.com/show_interview.php?id='):
         links.append(url)

### For loop for all the links in the players page

In [39]:
press_conferences = pd.DataFrame(columns=['date','question','answer'])
#dates = []
events = []
for link in links:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Getting Data
    #date = soup.find('h2').text
    event = soup.find('h1').text

    #dates.append(date)
    events.append(event)

    # print(date,'\t', event)


    dates_loop, answers_loop = sm.parse_interview(link)
    # Assuming dates_loop and answers_loop are complete
    

    bold_text = soup.find_all('b')
    questions = get_questions(bold_text) 
    stripped_questions = [question.strip('Q.') for question in questions]
    stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]
    num_to_fill = len(answers_loop) - len(stripped_questions)
    stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions
    temp_df = pd.DataFrame({'date': dates_loop, 'answer': answers_loop, 'question': stripped_questions})
    press_conferences = pd.concat([press_conferences, temp_df], ignore_index=True)
    print('Press conference from:',event)






Press conference from: LIV GOLF JEDDAH
Press conference from: LIV GOLF JEDDAH
Press conference from: LIV GOLF JEDDAH
Press conference from: LIV GOLF MAYAKOBA
Press conference from: LIV GOLF MAYAKOBA
Press conference from: LIV GOLF MAYAKOBA
Press conference from: LIV GOLF MAYAKOBA
Press conference from: LIV GOLF TEAM CHAMPIONSHIP MIAMI
Press conference from: LIV GOLF TEAM CHAMPIONSHIP MIAMI
Press conference from: LIV GOLF TEAM CHAMPIONSHIP MIAMI
Press conference from: LIV GOLF GREENBRIER
Press conference from: LIV GOLF GREENBRIER
Press conference from: LIV GOLF GREENBRIER
Press conference from: LIV GOLF ANDALUCIA
Press conference from: LIV GOLF ANDALUCIA
Press conference from: LIV GOLF WASHINGTON DC
Press conference from: MASTERS TOURNAMENT
Press conference from: MASTERS TOURNAMENT
Press conference from: MASTERS TOURNAMENT
Press conference from: MASTERS TOURNAMENT
Press conference from: LIV GOLF ORLANDO
Press conference from: LIV GOLF TUCSON
Press conference from: LIV GOLF MAYAKOBA
Pres

In [41]:
press_conferences2 = press_conferences.copy()

In [42]:
press_conferences2['player'] = [i for i, n in press_conferences2['answer']]
press_conferences2['answer'] = [n for i, n in press_conferences2['answer']]

press_conferences2['Ans LIV?'] =press_conferences2['answer'].apply(lambda x: 1 if 'liv' in x else 0)
press_conferences2['Ans PGA?'] =press_conferences2['answer'].apply(lambda x: 1 if 'pga' in x else 0)
press_conferences2['Q LIV?'] =press_conferences2['question'].apply(lambda x: 1 if 'liv' in x else 0)
press_conferences2['Q PGA?'] =press_conferences2['question'].apply(lambda x: 1 if 'pga' in x else 0)

In [44]:
press_conferences2.head(50)

,date,question,answer,player,Ans LIV?,Ans PGA?,Q LIV?,Q PGA?
0,"March 3, 2024",MODERATOR,"[yeah, it, feels, amazing, it's, a, super, spe...",joaquin niemann,0,0,0,0
1,"March 3, 2024","[you, looked, in, control, out, there, all, da...","[i, wouldn't, say, it, felt, that, way, at, th...",joaquín niemann,0,0,0,0
2,"March 3, 2024","[the, way, you're, playing, right, now, do, yo...","[i, mean, at, least, that's, what, i, think, i...",joaquin niemann,0,0,0,0
3,"March 3, 2024","[is, this, as, good, of, a, confidence, level,...","[i, believe, so, yeah, i, feel, like, it's, pr...",joaquin niemann,0,0,0,0
4,"March 3, 2024","[there, was, one, shot, i, think, it, was, the...","[oh, that, was, beautiful, yeah, that, hole, i...",joaquin niemann,0,0,0,0
5,"March 3, 2024","[you, talked, yesterday, about, an, adjustment...","[yeah, i, used, to, feel, a, lot, more, my, ri...",joaquin niemann,0,0,0,0
6,"March 3, 2024","[you, feel, like, you're, a, better, putter, now]","[it's, been, streaky, i've, been, having, good...",joaquin niemann,0,0,0,0
7,"March 3, 2024","[that, draw, is, working, so, nicely, at, the,...","[yeah, obviously, there, is, a, lot, more, i'l...",joaquin niemann,0,0,0,0
8,"March 3, 2024","[what, was, the, best, thing, about, today, yo...","[yeah, that, was, my, goal, today, was, to, tr...",joaquin niemann,0,0,0,0
9,"March 3, 2024","[we, saw, on, the, live, feed, that, you, were...","[yeah, gary's, role, was, huge, not, only, tod...",joaquin niemann,0,0,0,0



## HAY QUE PILLAR LAS FECHAS PARA LUEGO FILTRAR A PARTIR DE OCTUBRE DE 2020

### Habrá que checkear también si quien habla aparece dentro de los nombres en la conferencia de prensa, y luego checkear si es uno de nuestros jugadores seleccionados.
